#Generación de lenguaje natural
El objetivo de este notebook es mostrar mecanismos básicos de creación de un modelo completo del lenguaje natural que ayude a predecir las siguientes palabras, y de ese modo, generar texto.

In [1]:
import sklearn
import os, re, string, collections, random
import spacy
import nltk
import numpy, matplotlib
import pandas as pd

Creemos un corpus de NLTK

In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [3]:
from nltk.corpus import PlaintextCorpusReader
corpus_root="/content/drive/My Drive/corpus/libros"
corpusesp=PlaintextCorpusReader(corpus_root,".*", encoding="UTF-8")
"""para ver que archivos quedaron"""
corpusesp.fileids()


['candido-de-voltaire.txt',
 'cuentos_allan_poe.txt',
 'el-buscon.txt',
 'elquijote.txt',
 'fabulas_samaniego.txt',
 'juan_tenorio.txt',
 'libro_cocina.txt',
 'milyuna_t1.txt',
 'nuevo_testamento_valera.txt',
 'obras-escogidas_becquer.txt',
 'odisea.txt',
 'relacionhistoricasucesosdetupacamaru.txt',
 'tradiciones_peruanas_ricardo_palma.txt']

Y creemos el corpus en una lista

In [15]:
os.chdir("/content/drive/My Drive/corpus/libros")
documents = []
for f in os.listdir():
    if f[-4:] == '.txt':
        documents.append(f[:-4])



In [16]:
documents

['milyuna_t1',
 'relacionhistoricasucesosdetupacamaru',
 'tradiciones_peruanas_ricardo_palma',
 'elquijote',
 'cuentos_allan_poe',
 'juan_tenorio',
 'libro_cocina',
 'obras-escogidas_becquer',
 'candido-de-voltaire',
 'el-buscon',
 'nuevo_testamento_valera',
 'fabulas_samaniego',
 'odisea']

In [ ]:
len(documents)

13

In [17]:
contents = []
for document in documents:
    with open(document+'.txt', 'r', encoding="UTF-8") as f:
        contents.append(f.read())
len(contents)


13

In [18]:
for i in range(0,len(contents)):
    inicio=contents[i].find("EBOOK")
    final=contents[i].find("END OF")
    contents[i]=contents[i][inicio:final]

In [19]:
for i in range(len(contents)):
    contents[i] = re.sub("\""," ",contents[i])
    contents[i] = re.sub ("\n|\t"," ",contents[i])
    contents[i]=contents[i].lower()



Los generadores de lenguaje natural tiene como objetivo predecir, dado un texto anterior, el texto que viene después. Si bien su aplicación inicial es simplemente ser predictores de texto, se han convertido en herramienta fundamental para las aplicaciones modernas de NLP, pues generan una representación del lenguaje. Miremos:

In [20]:
import nltk 
from nltk import bigrams, trigrams

from collections import Counter, defaultdict

       
candido=corpusesp.words("candido-de-voltaire.txt")

pares=nltk.bigrams(candido)
generator=nltk.ConditionalFreqDist(pares)

##de pronto cambiar a 3 para hablar de predictor de texto"
def predictor(dist,palabra,num):
    for i in range(num):
        print(palabra,end=" ")
        palabra=dist[palabra].max()



In [11]:
#probar vieja, hizo, real#        
predictor(generator,"vieja",12)


vieja , y el señor baron , y el señor baron , 

In [12]:
generator["señor"]

FreqDist({'baron': 12, ',': 7, 'inquisidor': 4, 'abate': 4, 'Pococurante': 4, 'Panglós': 3, 'gobernador': 3, 'Martin': 3, 'de': 2, 'Don': 2, ...})

Ahora vamos a hacer un generador más complejo (basado en un ejemplo visto en analyticsvidtha). Creo un contenedor de modelo:

In [21]:
documents

['milyuna_t1',
 'relacionhistoricasucesosdetupacamaru',
 'tradiciones_peruanas_ricardo_palma',
 'elquijote',
 'cuentos_allan_poe',
 'juan_tenorio',
 'libro_cocina',
 'obras-escogidas_becquer',
 'candido-de-voltaire',
 'el-buscon',
 'nuevo_testamento_valera',
 'fabulas_samaniego',
 'odisea']

In [22]:
import nltk
nltk.download('punkt')
model = defaultdict(lambda: defaultdict(lambda: 0))
prueba3=nltk.word_tokenize(contents[3])
prueba3[1:100]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['don',
 'quijote',
 '*',
 '*',
 '*',
 'produced',
 'by',
 'an',
 'anonymous',
 'project',
 'gutenberg',
 'volunteer',
 '.',
 'text',
 'file',
 'corrections',
 'and',
 'new',
 'html',
 'file',
 'by',
 'joaquin',
 'cuenca',
 'abela',
 '.',
 'el',
 'ingenioso',
 'hidalgo',
 'don',
 'quijote',
 'de',
 'la',
 'mancha',
 'tasa',
 'yo',
 ',',
 'juan',
 'gallo',
 'de',
 'andrada',
 ',',
 'escribano',
 'de',
 'cámara',
 'del',
 'rey',
 'nuestro',
 'señor',
 ',',
 'de',
 'los',
 'que',
 'residen',
 'en',
 'su',
 'consejo',
 ',',
 'certifico',
 'y',
 'doy',
 'fe',
 'que',
 ',',
 'habiendo',
 'visto',
 'por',
 'los',
 'señores',
 'dél',
 'un',
 'libro',
 'intitulado',
 'el',
 'ingenioso',
 'hidalgo',
 'de',
 'la',
 'mancha',
 ',',
 'compuesto',
 'por',
 'miguel',
 'de',
 'cervantes',
 'saavedra',
 ',',
 'tasaron',
 'cada',
 'pliego',
 'del',
 'dicho',
 'libro',
 'a',
 'tres',
 'maravedís',
 'y',
 'medio',
 ';',
 'el']

Cuento las frecuencias de co-ocurrencias en trigramas

In [23]:
   for w1, w2, w3 in trigrams(prueba3, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1

Transformo en probabilidades

In [24]:
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

dict(model["el","caballo"])


{'de': 0.2608695652173913,
 'al': 0.043478260869565216,
 'a': 0.043478260869565216,
 'no': 0.08695652173913043,
 'y': 0.043478260869565216,
 'pegaso': 0.043478260869565216,
 ',': 0.08695652173913043,
 'quedó': 0.043478260869565216,
 'en': 0.08695652173913043,
 'relinche': 0.043478260869565216,
 'está': 0.043478260869565216,
 'se': 0.043478260869565216,
 'lleno': 0.043478260869565216,
 'mostraba': 0.043478260869565216,
 'con': 0.043478260869565216}

Primer modelo: con palabras máximas

In [25]:
def mln(pal1, pal2, num=10):
    for i in range(num):
        base=dict(model[pal1,pal2])
        print(pal1, end=" ")
        pal1=pal2
        pal2=max(base,key=base.get)


mln("el","caballo")


el caballo de madera , sobre todo , y , 

In [26]:
mln("don", "quijote")


don quijote , y , en el mundo , y 

Se crea un problema debido a los bucles o ciclos. Es mejor escoger una palabra aleatoria al menos.

In [27]:
text = ["el", "caballo"]

sentence_finished = False
 
while not sentence_finished:
  # select a random probability threshold  
  r = random.random()
  accumulator = .0

  for word in model[tuple(text[-2:])].keys():
      accumulator += model[tuple(text[-2:])][word]
      # select words that are above the probability threshold
      if accumulator >= r:
          text.append(word)
          break

  if text[-2:] == [None, None]:
      sentence_finished = True

  if len(text)==100:
      sentence_finished = True
 
print (' '.join([t for t in text if t]))


el caballo de madera que los venció sola una cosa quiero castigar a este salteador de caminos . -todo eso es así -dijo don quijote- . ven acá , digo que no debe de ser algún grandísimo bellaco , con todo esto , quiso hacer tirteafuera de la que será . encomiéndeme a sanchica , su primer intento , y aquella gallardía que conviene en caso de duda , le dijo : `` ámexi , cristiano , siendo forzoso que pregunten muchos : `` ninguno responda ; porque , siendo a todas partes , y liólas sobre rocinante , ni


O aun mejor, con palabras proporcionales a su probabilidad

In [28]:
text3="el"
text4="caballo"
nueva=[text3,text4]
i=0
   
while i<100:
    temporal=random.choices(population=list(model[text3,text4].keys()),weights=list(model[text3,text4].values()),k=1)
    nueva.append(temporal[0])
    text3=text4
    text4=temporal[0]    
    i=i+1
    
totalcreado=" ".join(nueva)
totalcreado    


'el caballo no se sabe que la tengo y tres , y tras todos éstos parece mejor un poeta , y le paseó todo , que es de moro -respondió don quijote- : vete adonde quisieres , que con lo que tan aborrecida tenía . otros las cuenten y las demás cosas que en este gobierno y sin segundo , por más señas , que de cuando en semejantes causas . -por esta figura que anda por esta vez se conoció haber corrido algo , por venir , ni hemos decantado de donde arguyo yo que se llamaba torralba , la túnica'